<a href="https://colab.research.google.com/github/LeGiT300/MastersModelTraining/blob/main/MastersProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# prompt: how do I upload a dataset from my computer instead
# df = pd.read_csv('D:/Smoke_IT/MasterProject/Data/ivr_maternal_childcare_large_conversations[1].csv')
# this is the file

from google.colab import files
uploaded = files.upload()

import io
df = pd.read_csv(io.BytesIO(uploaded['ivr_maternal_childcare_large_conversations[1].csv']))


In [30]:
df.head(10)

,conversation_id,role,text,intent,response_type
0,1,user,How do I introduce my baby to new foods?,nutrition,directive
1,1,assistant,Introduce one new food at a time and watch for...,nutrition,directive
2,2,user,How often should newborns eat?,newborn_care,directive
3,2,assistant,"Every 2-3 hours, including at night, whether b...",newborn_care,directive
4,3,user,How do I burp my baby properly?,newborn_care,directive
5,3,assistant,Hold your baby upright and gently pat their ba...,newborn_care,directive
6,4,user,How can I relieve teething pain?,baby_teething,directive
7,4,assistant,"Use chilled teething rings, massage gums, or o...",baby_teething,directive
8,5,user,What are the benefits of skin-to-skin contact?,newborn_care,informative
9,5,assistant,"It helps regulate the baby's temperature, hear...",newborn_care,informative


In [31]:
# Filter user questions and labels
questions = df[df['role'] == 'user']['text'].values
intents = df[df['role'] == 'user']['intent'].values

questions = [str(q) for q in questions]

In [32]:
print(intents)

['nutrition' 'newborn_care' 'newborn_care' ... 'labor_preparation'
 'baby_development' 'labor_signs']


In [33]:
# Tokenize questions
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") # Increased vocabulary size
tokenizer.fit_on_texts(questions)
sequences = tokenizer.texts_to_sequences(questions)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post') # Increased maxlen and added truncating


In [34]:
# Encode labels
intent_encoder = LabelEncoder()
response_type_encoder = LabelEncoder()

encoded_intents = intent_encoder.fit_transform(intents)
encoded_response_types = response_type_encoder.fit_transform(response_types)


In [35]:
intent_encoder = LabelEncoder()
encoded_intents = intent_encoder.fit_transform(intents)

X_train, X_test, y_train_intent, y_test_intent = train_test_split(
    padded_sequences,
    encoded_intents,
    test_size=0.2,
    random_state=42,
    stratify=encoded_intents # Stratify to ensure balanced class distribution in train/test splits
)

In [43]:
# Build model with improvements
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 64 # Increased embedding dimension
num_intent_classes = len(intent_encoder.classes_)
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=100),
    tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)), # Added Bidirectional LSTM
    tf.keras.layers.Bidirectional(LSTM(32)),
    Dropout(0.3), # Adjusted dropout rate
    Dense(64, activation='relu'),
    Dense(num_intent_classes, activation='softmax')
])

In [44]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [45]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # Implement early stopping

history = model.fit(
    X_train,
    y_train_intent,
    epochs=200, # Increased epochs, but early stopping will prevent overfitting
    batch_size=64, # Increased batch size
    validation_split=0.2,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_intent)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.4482 - loss: 1.7686 - val_accuracy: 1.0000 - val_loss: 0.0105
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9995 - loss: 0.0268 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 1.0000 - loss: 0.0082 - val_accuracy: 1.0000 - val_loss: 3.3252e-04
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9999 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 1.3687e-04
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 9.4190e-05
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 3.7380e-05
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9998 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 2.3029e-05
Epoch 8/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.99

In [52]:
# prompt: how to save the trained model locally on my pc. use model.export
# FileNotFoundError: Cannot find file: IVR.h5.zip

# ... (Your existing code)

# Save the model in h5 format
model.save("IVR.h5")

# Download the saved model to your local machine
from google.colab import files
files.download("IVR.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>